In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeCV, Ridge, LinearRegression
from sklearn.pipeline import Pipeline
import scipy
from tqdm import tqdm_notebook

In [ ]:
df_train = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/train.csv")
df_test = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/test.csv")
df_test_labels = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv")
df_test=df_test.merge(df_test_labels, how='inner', on='id')

df_total=pd.concat([df_train,df_test]).reset_index(drop=True)
df_total=df_total[df_total['toxic']!=-1]

df_total

In [ ]:
dict_mapping_kaggle={
    'severe_toxic':5,
    'obscene':1,
    'threat':4,
    'insult':2,
    'identity_hate':3
}


def convert_kaggle(dataframe):
    list_label=[]
    for index,row in tqdm_notebook(dataframe.iterrows(), total=len(dataframe)):
        if(row['toxic']==0):
            list_label.append(0)
        else:
            final_label=0
            count=0
            for col in list(dataframe.columns)[3:]:
                if(row[col]==1):
                    final_label+=dict_mapping_kaggle[col]
                    count+=1
            if(final_label==0):
                final_label=(1+2+3+4+5)/5
            else:
                final_label=final_label/count
            list_label.append(final_label/5)
    return list_label    
            
list_labels=convert_kaggle(df_total)
df_total['label']=list_labels
df_total = df_total.rename(columns={"comment_text": "text"})

# # df_total = df_total.head(1000)

In [ ]:

# df_total['severe_toxic'] = df_total.severe_toxic * 2
# # df['sum'] = df['toxic'] + df['severe_toxic'] + df['obscene'] + df['insult'] + df['threat'] + df['identity_hate']
# df_total['label'] = df_total.iloc[:, 2:9].sum(axis = 1)
# print(df_total['label'].max())
# df_total['label']  = df_total['label']/df_total['label'].max()

# df_total = df_total.rename(columns={"comment_text": "text"})

In [ ]:
df_total

In [ ]:
# print(len(df_total))
# len(df_total[df_total['label'] != 0])
# np.random.seed(10)

# remove_n = len(df_total[df_total['label'] == 0]) - len(df_total[df_total['label'] != 0])
# drop_indices = np.random.choice(df_total.index, remove_n, replace=False)
# df_total.drop(drop_indices, inplace = True)

In [ ]:
df_total

In [ ]:
import re
from collections import defaultdict

special_character_removal=re.compile(r'[^?!.,:a-z\d ]',re.IGNORECASE)
# regex to replace all numerics
replace_numbers=re.compile(r'\d+',re.IGNORECASE)
word_count_dict = defaultdict(int)
toxic_dict = {}

def clean_text(text, remove_stopwords=False, stem_words=False, count_null_words=True, clean_wiki_tokens=True):
    # Clean the text, with the option to remove stopwords and to stem words.
    # dirty words
    #text = text.lower()
    text = re.sub(r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)", "", text)
    text = re.sub(r"(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)){3}", "", text)
    
    if clean_wiki_tokens:
        # Drop the image
        text = re.sub(r"image:[a-zA-Z0-9]*\.jpg", " ", text)
        text = re.sub(r"image:[a-zA-Z0-9]*\.png", " ", text)
        text = re.sub(r"image:[a-zA-Z0-9]*\.gif", " ", text)
        text = re.sub(r"image:[a-zA-Z0-9]*\.bmp", " ", text)

        # Drop css
        text = re.sub(r"#([A-Fa-f0-9]{6}|[A-Fa-f0-9]{3})", " ",text)
        text = re.sub(r"\{\|[^\}]*\|\}", " ", text)
        
        # Clean templates
        text = re.sub(r"\[?\[user:.*\]", " ", text)
        text = re.sub(r"\[?\[user:.*\|", " ", text)        
        text = re.sub(r"\[?\[wikipedia:.*\]", " ", text)
        text = re.sub(r"\[?\[wikipedia:.*\|", " ", text)
        text = re.sub(r"\[?\[special:.*\]", " ", text)
        text = re.sub(r"\[?\[special:.*\|", " ", text)
        text = re.sub(r"\[?\[category:.*\]", " ", text)
        text = re.sub(r"\[?\[category:.*\|", " ", text)
    
   
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\?", " ? ", text)
    text = re.sub(r"\!", " ! ", text)
    text = re.sub(r"\"", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = replace_numbers.sub(' ', text)
    #text = special_character_removal.sub('',text)

    if count_null_words:
        text = text.split()
        for t in text:
            word_count_dict[t] += 1
        text = " ".join(text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)

    return (text)

df_total['text'] = [clean_text(text) for text in df_total['text']]

In [ ]:
pipeline = Pipeline(
    [
#         ("vect", TfidfVectorizer(analyzer = 'char_wb',  max_features = 5000)),
        ("vect", TfidfVectorizer(min_df= 3, max_df=0.5, analyzer = 'char_wb', ngram_range = (3,5),  max_features = 10000)),
        
        #("clf", RandomForestRegressor(n_estimators = 5, min_sample_leaf=3)),
        ("clf", Ridge()),
        #("clf",LinearRegression())
    ],verbose=True
)

In [ ]:
# Train the pipeline
pipeline.fit(df_total['text'], df_total['label'])

In [ ]:
df_val = pd.read_csv("../input/jigsaw-toxic-severity-rating/validation_data.csv")
df_val.drop_duplicates(subset=['less_toxic', 'more_toxic'], keep='first', inplace=True)
df_val.reset_index(inplace=True)


In [ ]:
df_val['less_toxic'] = [clean_text(text) for text in df_val['less_toxic']]
df_val['more_toxic'] = [clean_text(text) for text in df_val['more_toxic']]
p1 = pipeline.predict(df_val['less_toxic'])
p2 = pipeline.predict(df_val['more_toxic'])

In [ ]:
f'Validation Accuracy is { np.round((p1 < p2).mean() * 100,2)}'

In [ ]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(pipeline, open(filename, 'wb'))

In [ ]:
test_df = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")
# # test_df = test_df.head(10)

In [ ]:
test_df['text'] = [clean_text(text) for text in test_df['text']]

out = pipeline.predict(test_df['text'])
out

In [ ]:
from scipy.stats import rankdata


submission = pd.DataFrame({'comment_id': test_df['comment_id'], 'score':out})
submission['score'] = rankdata(submission['score'], method='ordinal')
submission.to_csv('submission.csv', index=False)

In [ ]:
submission